In [1]:
from hyperopt import hp, fmin, tpe

In [2]:
import gc
import os
import warnings

warnings.filterwarnings("ignore")
import itertools
import random

import joblib
import numpy as np
import pandas as pd
import scipy as sp

pd.set_option("display.max_rows", 500)
pd.set_option("display.max_columns", 500)
pd.set_option("display.width", 1000)
import gc
import warnings
from itertools import combinations

import lightgbm as lgb
from sklearn.model_selection import StratifiedKFold, train_test_split
from sklearn.preprocessing import LabelEncoder
from tqdm.auto import tqdm

warnings.filterwarnings("ignore")
import numpy as np
import pandas as pd
import scipy as sp

pd.set_option("display.max_rows", 500)
pd.set_option("display.max_columns", 500)
pd.set_option("display.width", 1000)
import itertools

from tqdm.auto import tqdm

In [13]:
import numpy as np
import pandas as pd
import lightgbm as lgb
from sklearn.model_selection import KFold
from hyperopt import hp, fmin, tpe
from numpy.random import RandomState
from sklearn.metrics import mean_squared_error

In [4]:
# ====================================================
# Read data
# ==================================================== 
def read_data():
    train = pd.read_parquet("./train_diff.parquet")
    test = pd.read_parquet("./test_diff.parquet")
    return train, test


In [5]:
train,test = read_data()

In [8]:
train.head(5)

,customer_ID,P_2_mean,P_2_std,P_2_min,P_2_max,P_2_last,D_39_mean,D_39_std,D_39_min,D_39_max,D_39_last,B_1_mean,B_1_std,B_1_min,B_1_max,B_1_last,B_2_mean,B_2_std,B_2_min,B_2_max,B_2_last,R_1_mean,R_1_std,R_1_min,R_1_max,R_1_last,S_3_mean,S_3_std,S_3_min,S_3_max,S_3_last,D_41_mean,D_41_std,D_41_min,D_41_max,D_41_last,B_3_mean,B_3_std,B_3_min,B_3_max,B_3_last,D_42_mean,D_42_std,D_42_min,D_42_max,D_42_last,D_43_mean,D_43_std,D_43_min,D_43_max,D_43_last,D_44_mean,D_44_std,D_44_min,D_44_max,D_44_last,B_4_mean,B_4_std,B_4_min,B_4_max,B_4_last,D_45_mean,D_45_std,D_45_min,D_45_max,D_45_last,B_5_mean,B_5_std,B_5_min,B_5_max,B_5_last,R_2_mean,R_2_std,R_2_min,R_2_max,R_2_last,D_46_mean,D_46_std,D_46_min,D_46_max,D_46_last,D_47_mean,D_47_std,D_47_min,D_47_max,D_47_last,D_48_mean,D_48_std,D_48_min,D_48_max,D_48_last,D_49_mean,D_49_std,D_49_min,D_49_max,D_49_last,B_6_mean,B_6_std,B_6_min,B_6_max,B_6_last,B_7_mean,B_7_std,B_7_min,B_7_max,B_7_last,B_8_mean,B_8_std,B_8_min,B_8_max,B_8_last,D_50_mean,D_50_std,D_50_min,D_50_max,D_50_last,D_51_mean,D_51_std,D_51_min,D_51_max,D_51_last,B_9_mean,B_9_std,B_9_min,B_9_max,B_9_last,R_3_mean,R_3_std,R_3_min,R_3_max,R_3_last,D_52_mean,D_52_std,D_52_min,D_52_max,D_52_last,P_3_mean,P_3_std,P_3_min,P_3_max,P_3_last,B_10_mean,B_10_std,B_10_min,B_10_max,B_10_last,D_53_mean,D_53_std,D_53_min,D_53_max,D_53_last,S_5_mean,S_5_std,S_5_min,S_5_max,S_5_last,B_11_mean,B_11_std,B_11_min,B_11_max,B_11_last,S_6_mean,S_6_std,S_6_min,S_6_max,S_6_last,D_54_mean,D_54_std,D_54_min,D_54_max,D_54_last,R_4_mean,R_4_std,R_4_min,R_4_max,R_4_last,S_7_mean,S_7_std,S_7_min,S_7_max,S_7_last,B_12_mean,B_12_std,B_12_min,B_12_max,B_12_last,S_8_mean,S_8_std,S_8_min,S_8_max,S_8_last,D_55_mean,D_55_std,D_55_min,D_55_max,D_55_last,D_56_mean,D_56_std,D_56_min,D_56_max,D_56_last,B_13_mean,B_13_std,B_13_min,B_13_max,B_13_last,R_5_mean,R_5_std,R_5_min,R_5_max,R_5_last,D_58_mean,D_58_std,D_58_min,D_58_max,D_58_last,S_9_mean,S_9_std,S_9_min,S_9_max,S_9_last,B_14_mean,B_14_std,B_14_min,B_14_max,B_14_last,D_59_mean,D_59_std,D_59_min,D_59_max,D_59_last,D_60_mean,D_60_std,D_60_min,D_60_max,D_60_last,D_61_mean,D_61_std,D_61_min,D_61_max,D_61_last,B_15_mean,B_15_std,B_15_min,B_15_max,B_15_last,S_11_mean,S_11_std,S_11_min,S_11_max,...,D_138_std,D_138_min,D_138_max,D_138_last,D_139_mean,D_139_std,D_139_min,D_139_max,D_139_last,D_140_mean,D_140_std,D_140_min,D_140_max,D_140_last,D_141_mean,D_141_std,D_141_min,D_141_max,D_141_last,D_142_mean,D_142_std,D_142_min,D_142_max,D_142_last,D_143_mean,D_143_std,D_143_min,D_143_max,D_143_last,D_144_mean,D_144_std,D_144_min,D_144_max,D_144_last,D_145_mean,D_145_std,D_145_min,D_145_max,D_145_last,B_30_count,B_30_last,B_30_nunique,B_38_count,B_38_last,B_38_nunique,D_114_count,D_114_last,D_114_nunique,D_116_count,D_116_last,D_116_nunique,D_117_count,D_117_last,D_117_nunique,D_120_count,D_120_last,D_120_nunique,D_126_count,D_126_last,D_126_nunique,D_63_count,D_63_last,D_63_nunique,D_64_count,D_64_last,D_64_nunique,D_66_count,D_66_last,D_66_nunique,D_68_count,D_68_last,D_68_nunique,P_2_diff1,D_39_diff1,B_1_diff1,B_2_diff1,R_1_diff1,S_3_diff1,D_41_diff1,B_3_diff1,D_42_diff1,D_43_diff1,D_44_diff1,B_4_diff1,D_45_diff1,B_5_diff1,R_2_diff1,D_46_diff1,D_47_diff1,D_48_diff1,D_49_diff1,B_6_diff1,B_7_diff1,B_8_diff1,D_50_diff1,D_51_diff1,B_9_diff1,R_3_diff1,D_52_diff1,P_3_diff1,B_10_diff1,D_53_diff1,S_5_diff1,B_11_diff1,S_6_diff1,D_54_diff1,R_4_diff1,S_7_diff1,B_12_diff1,S_8_diff1,D_55_diff1,D_56_diff1,B_13_diff1,R_5_diff1,D_58_diff1,S_9_diff1,B_14_diff1,D_59_diff1,D_60_diff1,D_61_diff1,B_15_diff1,S_11_diff1,D_62_diff1,D_65_diff1,B_16_diff1,B_17_diff1,B_18_diff1,B_19_diff1,B_20_diff1,S_12_diff1,R_6_diff1,S_13_diff1,B_21_diff1,D_69_diff1,B_22_diff1,D_70_diff1,D_71_diff1,D_72_diff1,S_15_diff1,B_23_diff1,D_73_diff1,P_4_diff1,D_74_diff1,D_75_diff1,D_76_diff1,B_24_diff1,R_7_diff1,D_77_diff1,B_25_diff1,B_26_diff1,D_78_diff1,D_79_diff1,R_8_diff1,R_9_diff1,S_16_diff1,D_80_diff1,R_10_diff1,R_11_diff1,B_27_diff1,D_81_diff1,D_82_diff1,S_17_di

In [16]:
train.shape

(458913, 1097)

Wrapper特征筛选

In [14]:
def feature_select_wrapper(train, test):
    """
    lgm特征重要性筛选函数
    :param train:训练数据集
    :param test:测试数据集
    :return:特征筛选后的训练集和测试集
    """
    
    # Part 1.划分特征名称，删除ID列和标签列
    print('feature_select_wrapper...')
    label = 'target'
    features = train.columns.tolist()
    features.remove('customer_ID')
    features.remove('target')

    # Step 2.配置lgb参数
    # 模型参数
    params_initial = {
        'num_leaves': 31,
        'learning_rate': 0.1,
        'boosting': 'gbdt',
        'min_child_samples': 20,
        'bagging_seed': 2020,
        'bagging_fraction': 0.7,
        'bagging_freq': 1,
        'feature_fraction': 0.7,
        'max_depth': -1,
        'metric': 'rmse',
        'reg_alpha': 0,
        'reg_lambda': 1,
        'objective': 'regression'
    }
    # 控制参数
    # 提前验证迭代效果或停止
    ESR = 30
    # 迭代次数
    NBR = 10000
    # 打印间隔
    VBE = 50
    
    # Part 3.交叉验证过程
    # 实例化评估器
    kf = KFold(n_splits=5, random_state=2020, shuffle=True)
    # 创建空容器
    fse = pd.Series(0, index=features)
    
    for train_part_index, eval_index in kf.split(train[features], train[label]):
        # 封装训练数据集
        train_part = lgb.Dataset(train[features].loc[train_part_index],
                                 train[label].loc[train_part_index])
        # 封装验证数据集
        eval = lgb.Dataset(train[features].loc[eval_index],
                           train[label].loc[eval_index])
        # 在训练集上进行训练，并同时进行验证
        bst = lgb.train(params_initial, train_part, num_boost_round=NBR,
                        valid_sets=[train_part, eval],
                        valid_names=['train', 'valid'],
                        early_stopping_rounds=ESR, verbose_eval=VBE)
        # 输出特征重要性计算结果，并进行累加
        fse += pd.Series(bst.feature_importance(), features)
    
    # Part 4.选择最重要的300个特征
    feature_select = ['customer_ID'] + fse.sort_values(ascending=False).index.tolist()[:300]
    print('done')
    return train[feature_select + ['target']], test[feature_select]

train_LGBM, test_LGBM = feature_select_wrapper(train, test)

In [15]:
train_LGBM, test_LGBM = feature_select_wrapper(train, test)

feature_select_wrapper...
[LightGBM] [Warning] Auto-choosing col-wise multi-threading, the overhead of testing was 3.305076 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 174913
[LightGBM] [Info] Number of data points in the train set: 367130, number of used features: 1087
[LightGBM] [Info] Start training from score 0.258571
Training until validation scores don't improve for 30 rounds
[50]	train's rmse: 0.264794	valid's rmse: 0.270304
[100]	train's rmse: 0.258962	valid's rmse: 0.267259
[150]	train's rmse: 0.255316	valid's rmse: 0.266489
[200]	train's rmse: 0.252134	valid's rmse: 0.266196
[250]	train's rmse: 0.24925	valid's rmse: 0.266087
[300]	train's rmse: 0.246529	valid's rmse: 0.265971
[350]	train's rmse: 0.244031	valid's rmse: 0.265949
Early stopping, best iteration is:
[321]	train's rmse: 0.245432	valid's rmse: 0.265896
[LightGBM] [Warning] Auto-choosing col-wise multi-threading, the overhead of testing was 2.807442 seconds.
You can

In [17]:
train_LGBM.shape

(458913, 302)

In [18]:
train_LGBM.head(5)

,customer_ID,P_2_last,D_39_last,B_4_last,B_3_last,R_1_last,B_4_std,D_43_last,S_3_last,P_2_min,D_46_last,D_48_diff1,B_5_last,P_2_mean,D_42_mean,B_2_last,S_3_mean,B_1_last,D_49_last,D_41_last,D_39_std,D_39_max,P_2_max,D_48_last,D_43_mean,P_3_last,B_9_last,B_9_mean,R_1_mean,S_23_last,B_9_diff1,B_11_last,D_42_max,S_11_mean,D_42_min,P_3_mean,D_46_mean,D_44_last,S_15_mean,S_9_mean,D_47_last,B_3_std,R_1_std,D_62_min,D_47_max,S_3_max,P_3_min,B_37_last,P_3_max,R_27_min,D_47_mean,D_49_std,D_46_min,S_9_last,S_5_diff1,D_44_std,B_7_last,D_61_diff1,D_121_min,R_1_max,D_70_mean,S_12_std,R_3_mean,B_3_diff1,D_50_mean,D_55_diff1,R_6_last,B_17_last,S_9_min,D_39_diff1,B_2_diff1,S_7_diff1,D_112_last,S_12_mean,D_50_max,D_39_mean,D_42_last,S_7_last,S_3_min,B_2_mean,D_43_max,S_24_last,P_3_std,S_5_last,B_25_diff1,B_40_max,B_9_max,D_51_mean,S_26_max,R_27_last,B_26_last,P_2_std,B_10_last,D_61_last,D_53_last,D_50_last,B_17_max,D_121_max,S_22_diff1,D_119_min,S_5_mean,S_12_min,B_14_diff1,S_25_diff1,S_3_std,D_52_mean,D_112_std,B_26_std,D_118_min,B_17_mean,D_105_max,B_40_last,S_12_diff1,D_43_diff1,D_55_std,B_2_min,R_3_last,D_46_max,D_46_diff1,S_25_max,S_26_mean,D_144_std,D_62_last,D_59_std,D_133_max,B_21_max,D_43_min,D_121_mean,D_66_last,S_5_min,B_1_diff1,B_28_diff1,B_40_std,D_62_mean,R_11_mean,D_48_std,B_7_diff1,S_8_last,P_2_diff1,D_52_min,B_5_min,D_52_last,S_7_min,B_19_std,R_1_diff1,B_21_mean,B_24_last,D_55_min,B_9_std,S_12_last,D_45_max,R_27_mean,D_65_max,S_5_max,D_43_std,B_17_diff1,D_77_min,S_26_std,S_9_max,B_26_diff1,R_2_last,D_102_std,R_11_last,D_48_max,D_75_std,D_61_std,B_11_diff1,S_3_diff1,B_18_diff1,S_7_std,B_37_diff1,S_26_last,B_18_last,D_102_diff1,S_24_diff1,D_41_std,R_6_mean,D_48_min,D_61_max,B_15_last,B_23_last,D_47_min,B_18_mean,S_23_std,D_102_max,D_61_min,D_69_mean,B_4_diff1,S_25_std,B_23_std,S_22_std,D_119_diff1,D_69_min,S_27_max,D_133_mean,S_27_std,S_23_max,B_21_std,S_5_std,B_21_last,D_45_mean,S_27_mean,B_22_mean,R_27_max,D_50_diff1,B_3_max,B_9_min,D_41_diff1,B_40_diff1,D_56_last,D_61_mean,S_23_mean,S_7_max,B_18_max,D_58_diff1,S_23_diff1,S_11_max,B_17_min,S_23_min,D_56_max,D_133_diff1,D_118_diff1,B_8_diff1,D_132_max,B_5_mean,D_60_last,D_133_std,S_24_std,B_13_diff1,S_7_mean,S_15_std,B_28_mean,S_27_diff1,S_25_last,S_27_last,B_28_min,D_71_mean,B_6_min,P_3_diff1,R_6_std,D_56_mean,D_65_std,D_53_std,B_18_min,D_52_diff1,S_8_mean,D_48_mean,D_142_std,S_11_std,B_8_mean,B_23_diff1,D_62_diff1,B_17_std,B_7_std,D_69_last,D_132_mean,D_105_std,B_15_std,D_69_max,S_12_max,R_27_diff1,D_71_max,S_22_last,D_55_mean,B_40_mean,S_17_std,D_56_min,B_24_std,D_74_mean,B_27_std,R_3_std,R_6_max,D_132_diff1,S_27_min,B_5_diff1,D_77_last,D_53_diff1,B_13_std,D_105_diff1,B_36_diff1,B_14_std,B_14_last,B_26_mean,S_11_last,B_24_min,D_130_diff1,B_36_std,D_77_mean,R_7_last,D_144_diff1,B_4_max,D_121_diff1,D_75_mean,D_71_last,B_5_max,B_27_diff1,D_41_max,B_5_std,D_46_std,D_62_max,B_6_last,target
0,0000099d6bd597052cdcda90ffabf56573fe9d7c79be5f...,0.934745,0,5,0.007174,0.006104,2.444250,NaN,0.135021,0.868580,0.420521,-0.044856,0.231717,0.933824,NaN,1.007647,0.113215,0.009382,-1,0.0,0.832050,3,0.960384,0.192376,NaN,0.629392,0.009535,0.006220,0.004509,0.131801,0.008471,0.010260,NaN,16.615385,NaN,0.680138,0.378074,0,4.692307,0.039818,0.539715,0.002942,0.003081,0.086805,0.542119,0.135021,0.581678,0.008676,0.741813,1.000246,0.532874,0.0,0.231009,0.007397,-0.015503,0.00000,0.058425,-0.000491,0.702280,0.009228,0.000000,0.126716,0.000000,-0.000280,0.150326,-0.009234,0.003945,NaN,0.007397,0.0,-0.000660,0.028320,1.0,0.247988,0.154326,0.230769,NaN,0.105671,0.098882,1.005086,NaN,0.936067,0.050671,0.034643,-0.003098,0.210060,0.009535,2.923077,0.027060,1.006130,0.001068,0.024194,0.326101,0.227637,NaN,0.153461,NaN,0.719791,-0.015536,0.236266,0.029112,0.085943,-0.019715,-0.007142,0.011670,0.204972,0.000000,0.003545,0.232120,NaN,1.073985,0.100454,-0.003377,NaN,0.068116,1.000242,0,0.519619,-0.005328,0.979136,0.012560,0.002598,0.446568,0.438529,0.009227,0.008716,NaN,0.711829,-1,0.007165,-0.010455,-0.01409